In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
# importing datasets
training = np.genfromtxt('./0000000000002419_training_ccpp_x_y_train.csv', delimiter=',')
testing = np.genfromtxt('./0000000000002419_test_ccpp_x_test.csv', delimiter=',')

In [3]:
training.shape

(7176, 5)

In [4]:
# prepare function  adds dummy vatiables of order 3 and adds intercept
def prepare(training):
    df = pd.DataFrame(training)
    X = len(df.columns)
    for i in range(X-1):
        for j in range(i, X-1):
            for k in range(j, X-1):
                for l in range(k, X-1):
                    df[f'{i}_{j}_{k}_{l}'] = df[i]*df[j]*df[k]*df[l]
    df['C'] = 1
    col_list = list(df.columns)
    col_list[X-1], col_list[-2] = col_list[-2], col_list[X-1]
    col_list[-1], col_list[-2] = col_list[-2], col_list[-1]
    df = df[col_list]
    training = np.array(df) 
    x = training[:,:-1]
    y = training[:, -1]
    return x, y
# Optimisation step
def step_grad(m, x, y, step_length):
    m_slope = np.zeros(len(x[0]))
    M = len(x[0])
    for i in range(len(x)):
        xi = x[i]
        yi = y[i]
    #calculating slope of m
        m_slope += (-2/M)*(yi-(m*xi).sum())*(xi)
    #calculting value of m
    return (m - (step_length*m_slope))
#error function for gradient descent
def cost(m, x, y):
    cost=0
    for i in range(len(x)):
        cost +=(1/len(x))*((y[i]-(m*x[i]).sum())**2)
    return(cost)

def gd(x, y, step_length, num_iter):
    m = np.zeros(len(x[0]))
    for i in range(num_iter):
        m = step_grad(m, x, y, step_length)
    #print(f'Cost for iteration {i}: {cost(m, x, y)}')
    
    return m
def run(x, y):
    num_iter = 500
    step_length = 0.000221
    m = gd(x, y, step_length, num_iter)
    return m

In [5]:
training = np.genfromtxt('./0000000000002419_training_ccpp_x_y_train.csv', delimiter=',')

x, y = prepare(training)
df = pd.DataFrame(x)
#feature scaling
scaler = preprocessing.StandardScaler()
scaler.fit(df.iloc[:,:-1])
df = scaler.transform(df.iloc[:,:-1])
for i in range(len(df)):
    x[i] = np.append(df[i], np.ones(1))

m = run(x, y)
n = len(x)
#predictions
pred =[]
for i in range(n):
    sigma = np.append(df[i,:], np.ones(1))
    pred.append(round(sum(sigma*m), 5))
# print(pred)

In [6]:
testing = np.genfromtxt('./0000000000002419_test_ccpp_x_test.csv', delimiter=',')

def test_prepare(training):
    df = pd.DataFrame(training)
    df['y'] = 1
    X = len(df.columns)
    for i in range(X-1):
        for j in range(i, X-1):
            for k in range(j, X-1):
                for l in range(k, X-1):
                    df[f'{i}_{j}_{k}_{l}'] = df[i]*df[j]*df[k]*df[l]
    df['C'] = 1
    col_list = list(df.columns)
    col_list[X-1], col_list[-2] = col_list[-2], col_list[X-1]
    col_list[-1], col_list[-2] = col_list[-2], col_list[-1]
    df = df[col_list]
    training = np.array(df) 
    x = training[:,:-1]
    return x

x = test_prepare(testing)
df = pd.DataFrame(x)
# print(df)
df = scaler.transform(df.iloc[:,:-1])
# print(pd.DataFrame(df))

pred1 = []
n = len(x)
for i in range(n):
    sigma = np.append(df[i,:], np.ones(1))
    pred1.append(round(sum(sigma*m), 5))
    
# print(pred1)

In [7]:
np.savetxt('./Predictions_CCPP_GD.csv',pred1, delimiter=',')